# A Mid-Infrared Analysis of Accreeting Supermassive Black Holes 
## Utilzing Diagnostic Tools to Identify Active Galactic Nuclei (AGN)
### Part 2: Evaluating the Completness and Reliability of the AGN Selection Criteria
As any selection can suffer from incompleteness and contamination, there is a need to choose a selection that is both reliable, and complete. In this notebook, we will evaluate the completeness and reliability of the AGN selection criteria.

Given the galaxies that were selected as AGN candidates, we will compare the selection diagnostic against a truth sameple of known AGN. These known AGN have been detected by the Code Investigating GALaxy Emission (CIGALE) fitting code.

To determine the completeness and reliability of the selection criteria, we will use the following definitions:

$$\text{Completeness} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN in the truth sample}}$$

$$\text{Reliability} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN selected by the criteria}}$$

In [344]:
# Begin by importing the required packages for the project
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits

#import seaborn as sns

In [345]:
# We now need to read in the dataframes that were created in the previous notebook for all fields
# We will also need to read in the truth samples that were created from the CIGALE code

# Choose a field to work with
field = 'CDFS'
# Read in the dataframes
selections_df = pd.read_csv(field+'_combined_selection.csv')



In [346]:

# Note, FITS files by virtue of their structure are big-endian, so we need to swap the bytes to use them in 
# Pandas Dataframes, as these are little-endian by default.

# Read in the truth samples and then create dataframes from the fits files
truth_sample = fits.open(field+'_truth.fits')
truth_df=pd.DataFrame(np.array(truth_sample[1].data).byteswap().newbyteorder()) # Byteswap so that Pandas can read it
truth_df.rename(columns={'id_1':'id'}, inplace=True) # Rename the ID column so that it matches the other dataframes

In [347]:
# Check the truth_df dataframe
truth_df.head()

,id,bayes.agn.total_dust_luminosity,bayes.agn.total_dust_luminosity_err,bayes.dust.luminosity,bayes.dust.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_star,bayes.stellar.m_star_err
0,5746,2.834282e+38,6.358570e+37,5.556047e+38,2.778023e+37,177.095988,8.854799,3.368743e+10,1.695740e+09
1,5860,3.109523e+37,4.202669e+37,7.505922e+38,5.678178e+37,203.540655,21.019983,4.616899e+10,8.537885e+09
2,5880,1.582654e+36,3.727870e+36,6.653951e+37,3.326975e+36,20.495446,1.024772,1.296424e+10,1.792094e+09
3,6020,1.065299e+38,1.218670e+37,2.165540e+38,2.411809e+37,52.781956,5.878316,2.554253e+10,2.845202e+09
4,6153,1.612261e+38,1.584376e+37,4.013813e+38,2.006907e+37,97.940061,4.897003,4.713909e+10,2.356954e+09


In [348]:
# bayes.agn.total_dust_luminosity <- AGN contribution
truth_df['bayes.agn.total_dust_luminosity']

# bayes.dust.luminosity <- stellar contribution
truth_df['bayes.dust.luminosity']

# filter the truth data based on error
truth_df = truth_df[truth_df['bayes.agn.total_dust_luminosity'] > 0]
truth_df = truth_df[truth_df['bayes.dust.luminosity'] > 0]



CIGALE uses a Bayesian approach with MCMC techniques. CIGALE allows us to analyse SED data, separatating the contributions of the AGN from the stellar-heated dust.  In the truth sample the luminosity from these two dust components can be used to derive a correct truth sample. A truth sample of AGN will be found if the luminosity contribution from the AGN is greater than 50% of the total luminosity of the system (AGN + Dust).

In [349]:
# To find the AGN we look at the values of the AGN's luminosity, and compare it against the
# total luminosity of that galaxy. If the lumniosty of the AGN is greater than 50% of the total
# luminosity of the entire galaxy, we have an AGN.


# This will be the AGN luminosity contribution
truth_df['agn contribution'] = truth_df['bayes.agn.total_dust_luminosity']/(truth_df['bayes.agn.total_dust_luminosity'] + truth_df['bayes.dust.luminosity'])

# Add a new column for known AGN
truth_df['Known AGN'] = np.where(truth_df['agn contribution'] > 0.5, 1, 0)

# This will be the AGN luminosity contribution
num_true_AGN = len(truth_df[truth_df['Known AGN'] == 1])

print("There are " + str(num_true_AGN) + " AGN that have been found by CIGALE in the " + field + " field.")

There are 106 AGN that have been found by CIGALE in the CDFS field.


Now that we know the amount of AGN, we need to use the ID from the truth sample, and append this to the AGN candidates. This will allow us to compare the selection criteria against the truth sample.If the ID from the truth sample is found in the AGN candidates, then we know that the AGN candidate is also an AGN in the truth sample and thus a positive diagnostic selection

As we have two selection criteria (Lacy, Messias) we will have two seperate positive diagnostic columns for each of the criteria.



In [350]:
# join the two dataframes
selections_df = selections_df.join(truth_df.set_index('id'), on='id')

In [351]:
# Create a new column for positive diagnostic, this will be set to zero initally
selections_df['Positive Lacy Selection'] = 0
selections_df['Positice Messias Selection'] = 0

# We now need to test against the AGN to see if we have a positive selection


# First for Lacy, then Messias
selections_df['Positive Lacy Selection'] = np.where((selections_df['Known AGN'] == 1) & (selections_df['Lacy Selection'] == 1), 1, 0)
selections_df['Positive Messias Selection'] = np.where((selections_df['Known AGN'] == 1) & (selections_df['Messias Selection'] == 1), 1, 0)

In [352]:
# Now we can do a comparison to determine the reliability and completeness of the selections
selections_df['Positive Lacy Selection'].value_counts()[1]
selections_df['Positive Messias Selection'].value_counts()[1]

25

We now can calculate the completeness of the of the selection, we do this using the criteria below

$$\text{Completeness} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN in the truth sample}}$$

In [353]:
def calculateCompleteness(df, diagnostic):
    # Calculate the completeness of the selection
    # Completeness = Positive Diagnostic / Known AGN
    # Positive Diagnostic = AGN that are selected by CIGALE, and as AGN by the selection diagnostic
    # Known AGN = AGN selected by CIGALE
    positive_selection = df['Positive '+ diagnostic + " Selection"].value_counts()[1]
    known_AGN = df['Known AGN'].value_counts()[1]
    return positive_selection/known_AGN

# Calculate the completeness of the Lacy selection
lacy_completeness = calculateCompleteness(selections_df, 'Lacy')

# Calculate the completeness of the Messias selection
messias_completeness = calculateCompleteness(selections_df, 'Messias')

We now can calculate the reliability of the of the selection, we do this using the criteria below

$$\text{Reliability} = \frac{\text{Number of AGN selected by the criteria that are also AGN in the truth sample}}{\text{Number of AGN selected by the criteria}}$$


In [354]:
def calculateReliability(df, diagnostic):
    # Calculate the reliability of the selection
    # Reliability = Positive Diagnostic / Positive Selection
    # Positive Diagnostic = AGN that are selected by CIGALE, and as AGN by the selection diagnostic
    # Positive Selection = AGN that are selected by the selection diagnostic
    positive_selection = df['Positive '+ diagnostic + " Selection"].value_counts()[1]
    
    positive_diagnostic = df[diagnostic + ' Selection'].value_counts()[1]
    return positive_selection/positive_diagnostic

lacy_reliability = calculateReliability(selections_df, 'Lacy')
messias_reliability = calculateReliability(selections_df, 'Messias')
selections_df


,id,x,y,ra,dec,SEflags,iso_area,fap_Ksall,eap_Ksall,apcorr,...,bayes.dust.luminosity_err,bayes.sfh.sfr,bayes.sfh.sfr_err,bayes.stellar.m_star,bayes.stellar.m_star_err,agn contribution,Known AGN,Positive Lacy Selection,Positice Messias Selection,Positive Messias Selection
0,5746,4778.412,2191.809,53.067131,-27.883856,2,362.0,2.698493,0.095665,1.031906,...,2.778023e+37,177.095988,8.854799,3.368743e+10,1.695740e+09,0.337803,0,0,0,0
1,5860,4916.620,2228.198,53.060615,-27.882338,3,247.0,8.315286,0.102656,1.075718,...,5.678178e+37,203.540655,21.019983,4.616899e+10,8.537885e+09,0.039780,0,0,0,0
2,5880,4918.070,2205.127,53.060547,-27.883299,3,134.0,3.127875,0.107667,1.095818,...,3.326975e+36,20.495446,1.024772,1.296424e+10,1.792094e+09,0.023233,0,0,0,0
3,6020,4722.533,2272.799,53.069767,-27.880484,3,341.0,7.114190,0.083399,1.050610,...,2.411809e+37,52.781956,5.878316,2.554253e+10,2.845202e+09,0.329728,0,0,0,0
4,6153,4741.378,2293.590,53.068878,-27.879616,3,293.0,3.878256,0.081004,1.049544,...,2.006907e+37,97.940061,4.897003,4.713909e+10,2.356954e+09,0.286569,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,30561,3161.745,7440.671,53.143299,-27.665165,3,251.0,6.202888,0.186485,1.065775,...,1.211326e+36,7.495037,0.393237,2.390450e+10,1.195225e+09,0.454833,0,0,0,0
882,30647,3094.975,7607.521,53.146439,-27.658211,2,266.0,8.943708,0.196572,1.064004,...,7.678264e+37,123.907630,30.253412,1.518658e+10,1.851144e+09,0.040666,0,0,0,0
883,30724,3584.657,7482.463,53.123402,-27.663424,0,25.0,0.674413,0.181990,1.377135,...,1.114254e+36,1.393609,0.276501,2.125778e+08,3.290448e+08,0.938122,1,1,0,1
884,30734,2790.194,7498.273,53.160778,-27.662760,0,122.0,4.020537,0.163558,1.110898,...,4.653180e+37,21.438396,12.942402,9.670290e+09,4.472081e+09,0.027463,0,0,0,0


## Conclusion
Bringing this all together we can see both the completeness and relaibility of each of the diagnostics. Below we see the outputs of the completeness and reliability tests for our Lacy and Messias diagnostics. 

In [355]:
print("The completeness of the Lacy selection is " + str(round(lacy_completeness*100, 2))+"% in the " + field + " field.")
print("The reliability of the Lacy selection is " + str(round(lacy_reliability*100, 2))+"% in the " + field + " field.")
print("\n")
print("The completeness of the Messias selection is " + str(round(messias_completeness*100, 2))+"% in the " + field + " field.")
print("The reliability of the Messias selection is " + str(round(messias_reliability*100, 2))+"% in the " + field + " field.")

The completeness of the Lacy selection is 95.28% in the CDFS field.
The reliability of the Lacy selection is 11.56% in the CDFS field.


The completeness of the Messias selection is 23.58% in the CDFS field.
The reliability of the Messias selection is 42.37% in the CDFS field.
